# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
import os
import time
# Set the working directory to the location of the `.env` file
os.chdir('../../02_activities/assignments')

# Load env variables from the `.env` file
from dotenv import load_dotenv
load_dotenv() 


%reload_ext dotenv
%dotenv



In [5]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:

import os
from glob import glob


PRICE_DATA = os.getenv('PRICE_DATA')
print(f"PRICE_DATA directory: {PRICE_DATA}")


parquet_files = glob(os.path.join(PRICE_DATA, '*/*/*.parquet'))
print(f"Parquet files found: {parquet_files[:5]}")  # Print the first few files


dd_px = dd.read_parquet(parquet_files,npartitions=len("ticker")).set_index("ticker")



PRICE_DATA directory: ../../05_src/data/prices/
Parquet files found: ['../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet', '../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [14]:
# Write your code below.
import numpy as np

# Create new features

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))


/var/folders/v2/3kvz36y16hg4ry1svvl61dmm0000gn/T/ipykernel_72596/2916511836.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [18]:
# Convert to pandas DataFrame
pd_feat = dd_feat.compute()

# Calculate 10 day rolling average return
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()

print(pd_feat.tail())

             Date        Open        High         Low       Close   Adj Close  \
ticker                                                                          
HII    2024-10-21  262.820007  263.459991  259.769989  261.239990  261.239990   
HII    2024-10-22  260.690002  260.690002  255.500000  257.109985  257.109985   
HII    2024-10-23  257.000000  257.269989  253.070007  256.000000  256.000000   
HII    2024-10-24  255.149994  257.559998  254.050003  254.699997  254.699997   
HII    2024-10-25  257.239990  258.470001  252.550003  253.830002  253.830002   

        Volume       sector            subsector  year  Close_lag_1  \
ticker                                                                
HII     185700  Industrials  Aerospace & Defense  2024   261.529999   
HII     175700  Industrials  Aerospace & Defense  2024   261.239990   
HII     196100  Industrials  Aerospace & Defense  2024   257.109985   
HII     172400  Industrials  Aerospace & Defense  2024   256.000000   
HII   

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No
It's better to do the calculation in Dask to avoid loading all the data into memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.